In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pickle

In [3]:
train_dir = "/content/drive/MyDrive/Cattle Classification/data/train"
val_dir = "/content/drive/MyDrive/Cattle Classification/data/val"
test_dir = "/content/drive/MyDrive/Cattle Classification/data/test"

IMG_SIZE = 224

In [6]:
# data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 15,
    horizontal_flip = True,
    width_shift_range = 0.10,
    height_shift_range = 0.10,
    zoom_range = 0.2,
    fill_mode = "nearest"
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True
)

# no augmentation
val_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = False
)

# no augmentation
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = False
)

Found 965 images belonging to 5 classes.
Found 119 images belonging to 5 classes.
Found 124 images belonging to 5 classes.


In [7]:
base_model = DenseNet121(include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(5, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# compile with Adam optimizer
model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

if __name__ == "__main__":

  history = model.fit(
    train_data,
    validation_data = val_data,
    epochs = 15,
    verbose = 1
  )

  # save trained model
  model.save("/content/drive/MyDrive/saved_models/densenet121.h5")

  with open("/content/drive/MyDrive/saved_models/history_densenet121.pkl", "wb") as f:
    pickle.dump(history.history, f)

test_loss, test_acc = model.evaluate(test_data, verbose=1)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 357s 11s/step - accuracy: 0.3650 - loss: 1.8181 - val_accuracy: 0.7395 - val_loss: 0.7893
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 484ms/step - accuracy: 0.6836 - loss: 0.8920 - val_accuracy: 0.8067 - val_loss: 0.6405
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 485ms/step - accuracy: 0.7653 - loss: 0.6927 - val_accuracy: 0.8319 - val_loss: 0.5364
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 509ms/step - accuracy: 0.7833 - loss: 0.6589 - val_accuracy: 0.8319 - val_loss: 0.5003
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 542ms/step - accuracy: 0.8049 - loss: 0.5301 - val_accuracy: 0.8487 - val_loss: 0.4440
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 500ms/step - accuracy: 0.8226 - loss: 0.4793 - val_accuracy: 0.8571 - val_loss: 0.4215
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 497ms/step - accuracy: 0.8529 - loss: 0.4866 - val_accuracy: 0.8403 - val_loss: 0.4264
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 507ms/step - accuracy: 0.8237 - loss: 0.4866 - val_accur

4/4 ━━━━━━━━━━━━━━━━━━━━ 34s 11s/step - accuracy: 0.8790 - loss: 0.4407
Test Loss: 0.3509 | Test Acc: 0.9032
